In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re


global_path = './outputs'
all_result_paths = glob.glob(global_path + '/mod3*_exp/**/result.csv', recursive=True)
all_result_paths = [p for p in all_result_paths if 'nc' not in p]
result_summary = {}
n_layers = 6
for result_path in all_result_paths:
    result = pd.read_csv(result_path)
    top_test_result = result['test_acc'].max()
    pe_type = 'NoPE' if 'nope' in result_path else 'Original'
    pattern = r"res=\[(.*?)\]"
    residual_list = re.findall(pattern, result_path)[0]
    residual_list = [int(num) for num in residual_list.split(", ")]
    missing_rc = [i for i in range(n_layers) if i not in residual_list]
    missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


    if pe_type not in result_summary:
        result_summary[pe_type] = {}
    if missing_rc_str not in result_summary[pe_type]:
        result_summary[pe_type][missing_rc_str] = []
    result_summary[pe_type][missing_rc_str].append(top_test_result)
    if pe_type == 'Original' and missing_rc_str == '{5}':
        print(result_path, pe_type, missing_rc_str, top_test_result)


./outputs/mod3_residual_exp/mod3_sd240_T2403251700_res=[0, 1, 2, 3, 4]/mod3_sd240_T2403251700_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0
./outputs/mod3_residual_exp/mod3_sd241_T2403252050_res=[0, 1, 2, 3, 4]/mod3_sd241_T2403252050_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0
./outputs/mod3_residual_exp/mod3_sd242_T2403260040_res=[0, 1, 2, 3, 4]/mod3_sd242_T2403260040_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0
./outputs/mod3_residual_exp/mod3_sd243_T2403260426_res=[0, 1, 2, 3, 4]/mod3_sd243_T2403260426_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0
./outputs/mod3_residual_exp/mod3_sd244_T2403260809_res=[0, 1, 2, 3, 4]/mod3_sd244_T2403260809_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0


In [4]:
from ast import literal_eval
mean_results = {}
for pe_type in result_summary:
    cur_type_dict = result_summary[pe_type].copy()
    for missing_rc in cur_type_dict:
        cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
    mean_results[pe_type] = cur_type_dict
df = pd.DataFrame(mean_results)
missing0 = ['{}']
missing1 = [str({i}) for i in range(n_layers)]
missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
from IPython.display import display
display(df.loc[missing0,:].T)
display(df.loc[missing1,:].T)
display(df.loc[missing2,:].T)
display(df.loc[missing3,:].T)

,{}
Original,5
NoPE,5


,{0},{1},{2},{3},{4},{5}
Original,5,5,5,5,5,5
NoPE,5,5,5,5,5,5


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original,5,5,5,5,5
NoPE,5,5,5,5,5


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original,5,5,5,5
NoPE,5,5,5,5


In [ ]:
from ast import literal_eval
from collections import OrderedDict
final_results = OrderedDict()
for pe_type in {'Original', 'NoPE'}:
    cur_type_dict = result_summary[pe_type].copy()
    final_dict = {
        'min': cur_type_dict.copy(),
        'max': cur_type_dict.copy(),
        'avg.': cur_type_dict.copy(),
    }
    for missing_rc in cur_type_dict:
        final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
        final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
        final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

    for k in final_dict:
        final_results[pe_type+' ('+k+')'] = final_dict[k]
df = pd.DataFrame(final_results)
missing1 = [str({i}) for i in range(n_layers)]
missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
from IPython.display import display
r1 = df.loc[missing1,:].T
r2 = df.loc[missing2,:].T
r3 = df.loc[missing3,:].T
r4 = df.loc[missing4,:].T
r5 = df.loc[missing5,:].T
r0 = df.loc[['{}'],:].T
display(r0)
display(r1)
display(r2)
display(r3)
display(r4)
display(r5)

# write all 3 to the same csv file
all_r = pd.concat([r0, r1, r2, r3], axis=1)
all_r.to_csv('all_r_mean.csv')


In [5]:
# This is causal trained on mod3, 5 seeds

,{}
NoPE (min),100.000
NoPE (max),100.000
NoPE (avg.),100.000
Original (min),99.760
Original (max),100.000
Original (avg.),99.858


,{0},{1},{2},{3},{4},{5}
NoPE (min),98.930,79.780,98.440,99.980,98.620,100.0
NoPE (max),99.990,99.060,100.000,100.000,100.000,100.0
NoPE (avg.),99.590,88.108,99.684,99.996,99.720,100.0
Original (min),98.190,89.130,98.800,99.200,98.810,100.0
Original (max),100.000,100.000,100.000,100.000,100.000,100.0
Original (avg.),99.298,96.900,99.746,99.836,99.762,100.0


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (min),94.920,96.140,98.570,99.980,99.990
NoPE (max),100.000,99.930,100.000,100.000,100.000
NoPE (avg.),98.944,98.398,99.378,99.996,99.998
Original (min),99.520,98.250,99.930,99.670,99.710
Original (max),100.000,99.760,100.000,100.000,100.000
Original (avg.),99.824,99.058,99.986,99.928,99.906


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (min),99.940,79.720,77.960,33.350
NoPE (max),100.000,94.470,99.150,33.990
NoPE (avg.),99.974,89.012,92.662,33.670
Original (min),99.090,88.960,82.210,33.640
Original (max),100.000,94.820,98.740,33.990
Original (avg.),99.498,90.902,94.848,33.776


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
NoPE (min),85.140,33.420,33.350
NoPE (max),94.920,34.490,33.990
NoPE (avg.),90.644,33.832,33.658
Original (min),63.880,33.480,33.510
Original (max),96.240,33.910,33.990
Original (avg.),78.078,33.708,33.750


,"{0,1,2,3,4}","{1,2,3,4,5}"
NoPE (min),33.580,33.350
NoPE (max),33.980,33.990
NoPE (avg.),33.736,33.658
Original (min),33.580,33.590
Original (max),33.910,33.990
Original (avg.),33.746,33.764


In [8]:
# This is non-causal trained on mod3, 2 seeds

,{}
NoPE (min),89.940
NoPE (max),95.460
NoPE (avg.),92.700
Original (min),98.540
Original (max),99.990
Original (avg.),99.265


,{0},{1},{2},{3},{4},{5}
NoPE (min),95.25,89.680,99.040,91.610,98.870,92.970
NoPE (max),98.77,99.400,99.480,100.000,100.000,95.260
NoPE (avg.),97.01,94.540,99.260,95.805,99.435,94.115
Original (min),88.77,87.210,99.740,100.000,100.000,99.020
Original (max),94.45,100.000,99.890,100.000,100.000,99.950
Original (avg.),91.61,93.605,99.815,100.000,100.000,99.485


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (min),90.920,92.180,97.400,93.68,91.310
NoPE (max),93.870,95.380,100.000,99.18,99.450
NoPE (avg.),92.395,93.780,98.700,96.43,95.380
Original (min),93.930,91.190,98.610,100.00,98.130
Original (max),98.590,94.680,100.000,100.00,99.500
Original (avg.),96.260,92.935,99.305,100.00,98.815


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (min),93.130000,83.570,99.490,95.48
NoPE (max),98.260000,93.940,100.000,100.00
NoPE (avg.),95.695000,88.755,99.745,97.74
Original (min),93.510000,83.460,96.980,100.00
Original (max),96.250000,90.990,100.000,100.00
Original (avg.),95.326667,87.225,98.490,100.00


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
NoPE (min),83.090,86.76,98.270
NoPE (max),87.200,92.12,99.000
NoPE (avg.),85.145,89.44,98.635
Original (min),86.570,82.86,33.810
Original (max),87.630,90.44,97.480
Original (avg.),86.960,86.60,75.460


,"{0,1,2,3,4}","{1,2,3,4,5}"
NoPE (min),82.58,36.010000
NoPE (max),86.98,90.720000
NoPE (avg.),84.78,63.365000
Original (min),85.71,81.720000
Original (max),91.76,89.060000
Original (avg.),88.68,85.413333
